In [2]:
import csv
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pickle

Using TensorFlow backend.


In [13]:
MergedFullDS = pd.read_csv("MergedDSFULL.csv")

C:\Users\Kasutaja\Anaconda3\envs\myEnv\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
MergedFullDS.head()

,text_ID,start,end,object,value,unit,min,max,text
0,46809,0,6,RR,130,NaN,NaN,NaN,RR 130/80 mmHg.
1,43588,2421,2433,ALP,54,U/L,NaN,NaN,UURINGUD JA PROTSEDUURID \nLÄBIVAATUS: Objekt...
2,43050,720,735,HCT,40,%,NaN,41.2,UURINGUD JA PROTSEDUURID \n|Uuring|24.04|25.0...
3,45389,165,171,RR,110,NaN,NaN,NaN,"Kergelt liigkaaluline. Nahk, limaskestad taval..."
4,42960,2314,2332,HCT,45,%,40.0,50.0,UURINGUD JA PROTSEDUURID \nObjektiivselt: Nor...


In [5]:
len(set(MergedFullDS.object))

2125

In [14]:
len(set(MergedFullDS.text_ID))

301106

In [9]:
MergedDS = pd.read_csv("MergedDS.csv")

In [10]:
MergedDS.head()

,text_ID,start,end,object,value,unit,min,max,text
0,46809,0,6,RR,130.0,NaN,NaN,NaN,RR 130/80 mmHg.
1,45389,165,171,RR,110.0,NaN,NaN,NaN,"Kergelt liigkaaluline. Nahk, limaskestad taval..."
2,47482,0,5,RR,142.0,NaN,NaN,NaN,"RR142/89mmHg, HR 76 x min, p167cm, k89kg. EKG..."
3,53393,0,5,RR,143.0,NaN,NaN,NaN,"RR143/87mmHg, HR60 x min, p180cm, k93kg"
4,53571,0,6,RR,140.0,NaN,NaN,NaN,RR 140/74 mm Hg Cor fr. 62 UH uuringul kaela a...


In [12]:
set(MergedDS.object)

{'Pikkus', 'RR', 'eGFR'}

In [21]:
pd.set_option('display.max_colwidth', -1)


C:\Users\Kasutaja\Anaconda3\envs\myEnv\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [36]:
print(MergedDS[(MergedDS["object"] == 'RR' )]["text"][150:180])

162    RR 120/90 mmHg.fr.51x/min.ebaregulaarne,SpO2 98%,kaal 91 kg.pikkus 175 cm.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [5]:
DS = pd.read_csv("ObjectSubset150SentenceLength.csv")

In [6]:
DS['tag'].fillna('text', inplace=True)

In [7]:
words = list(set(DS["word"].values))
n_words = len(words)
n_words

27741

In [40]:
tags = list(set(DS["tag"].values))
n_tags = len(tags); n_tags

2

In [6]:
Chars_DS = pd.read_csv("CharsDSDULL.csv")
Chars_DS.head()

,text_ID,char,tag
0,46809,R,object
1,46809,R,object
2,46809,1,object
3,46809,3,object
4,46809,0,object


In [7]:
len(Chars_DS)

979964

In [8]:
chars = set([w_i for w in words for w_i in w])
n_chars = len(chars)
print(n_chars)


104


In [8]:
class SentenceGetter(object):
    def __init__(self, dataset):
        self.n_sent = 1
        self.dataset = dataset
        self.empty = False
        agg_func = lambda s: [(w, t) for w,t in zip(s["word"].values.tolist(),
                                                        s["tag"].values.tolist())]
        self.grouped = self.dataset.groupby("text_ID").apply(agg_func)
        self.sentences = [s for s in self.grouped]
getter = SentenceGetter(DS)
sentences = getter.sentences

In [9]:
sentences[1]

[('Pt.', 'text'),
 ('tehtud', 'text'),
 ('3', 'text'),
 ('plasmaferees,', 'text'),
 ('talus', 'text'),
 ('hästi.', 'text'),
 ('Eemaldatud', 'text'),
 ('700', 'text'),
 ('ml', 'text'),
 ('plasmat.', 'text'),
 ('RR', 'object'),
 ('112/60', 'object'),
 ('mmHg.', 'text'),
 ('Tgasi', 'text'),
 ('22.10.09.', 'text')]

In [10]:
LengthsList = []
for i in range(len(sentences)):
    LengthsList.append(len(sentences[i]))

In [11]:
np.median(LengthsList)

10.0

In [12]:
len(sentences)

13643